In [1]:
# syft absolute
import syft as sy
from syft import ActionObject
from syft.client.syncing import compare_states
from syft.client.syncing import resolve

kj/filesystem-disk-unix.c++:1734: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft


In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=8760532393be4571be9a4d8edf910dd4 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=4d367f4a86f346aa8f77e73d3507d3be in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: Low side Domain> as <newuser@openmined.org>


# create datasets

In [5]:
# third party
import numpy as np

In [6]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

  0%|                                                             | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.38it/s]


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
# private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    id=dataset_high.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=ActionObject.empty(data_node_id=client_high.id),
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00, 14.07it/s]


Uploading: numpy-data


# Make Requests

In [8]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    print("Computing mean...")
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/acd794ea6c01458f802ad6466329b507.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=2 image_uid=5342b72f93824517ac3d1ee062e7aa86 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_1391: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_1391
> /Users/koen/workspace/PySyft/packages/syft/src/syft/service/request/request_service.py(216)apply()
    215                     ipdb.set_trace()
--> 216                     mark_as_read = context.node.get_service_method(
    217                         NotificationService.mark_as_read

ipdb> c
Computing mean...
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

In [9]:
client_low_ds.code.request_code_execution(compute_mean)

syft.service.request.request.Request

## Sync to high side

In [10]:
low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

SyftInfo: Node Landed!


In [11]:
low_state

syft.service.sync.sync_state.SyncState

In [12]:
diff_state = compare_states(low_state, high_state)

In [13]:
diff_state

syft.service.sync.diff_state.NodeDiff

In [14]:
resolved_state_low, resolved_state_high = resolve(diff_state, decision="low")

LOW SIDE STATE:

―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

REQUEST #83ddf1bc70ee434880e0af32ef29c7be:
request_time: 2024-03-04 19:06:02
updated_at: None
status: RequestStatus.PENDING
changes: [
‎ ‎ ‎ ‎ syft.service.request.request.UserCodeStatusChange
]
requesting_user_verify_key: 3307a8cb2c7b2ad85028d900964e73c88df25ff6eb1f53bd37668c82d3097119

    ―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODE #222a6f2ca259444ca94a4c719952ea58:
    service_func_name: compute_mean
    input_owners: [
    ‎ ‎ ‎ ‎ test_l
    ]
    code_status: [
    ‎ ‎ ‎ ‎ Node: test_l, Status: pending
    ]
    worker_pool_name: default-pool



HIGH SIDE STATE:

No high side changes.

Decision: Syncing 2 objects from low side


LOW SIDE STATE:

―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #222a6f2ca259444ca94a4c719952ea58:
service_

In [15]:
print("Resolved state low side")
print(resolved_state_low)
print()
print("Resolved state high side")
print(resolved_state_high)

Resolved state low side
ResolvedSyncState(
  create_objs=[],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)

Resolved state high side
ResolvedSyncState(
  create_objs=[syft.service.request.request.Request, syft.service.code.user_code.UserCode, {NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')}],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)


In [16]:
client_low.apply_state(resolved_state_low)

SyftSuccess: Synced 0 items

In [17]:
client_high.apply_state(resolved_state_high)

SyftSuccess: Synced 3 items

# Run code high and sync back result

In [18]:
data_high = client_high.datasets[0].assets[0]

In [19]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = 64bd0a0dcf0146f084a8da0ef89d3791
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [20]:
# wait for the result
job_high.wait().get()

04/03/24 19:06:10 FUNCTION LOG (64bd0a0dcf0146f084a8da0ef89d3791): Computing mean...


17.0

In [21]:
job_info = job_high.info(public_metadata=True, result=True)

request = client_high.requests[0]
result_obj = job_high.result

In [22]:
# syft absolute
from syft import SyftError
from syft import SyftSuccess

# Accepting the result directly gives an error
accept_res = request.accept_by_depositing_result(result_obj)
assert isinstance(accept_res, SyftError)

In [23]:
accept_res = request.accept_by_depositing_result(job_info)

assert isinstance(accept_res, SyftSuccess)
accept_res

Approving request for domain test_h
returning existing job
setting permission
None
None
Approving request for domain test_h
ADDING PERMISSION [READ: 059d580a4b904c60968e1a8a0360551f as 3307a8cb2c7b2ad85028d900964e73c88df25ff6eb1f53bd37668c82d3097119] 059d580a4b904c60968e1a8a0360551f
New result 17.0
Job(64bd0a0dcf0146f084a8da0ef89d3791) Setting new result 059d580a4b904c60968e1a8a0360551f -> 059d580a4b904c60968e1a8a0360551f


SyftSuccess: Request submitted for updating result.

In [24]:
accept_res

SyftError: Exception calling request.apply. Traceback (most recent call last):
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/node/node.py", line 1211, in handle_api_call_with_unsigned_result
    result = method(context, *api_call.args, **api_call.kwargs)
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/service.py", line 357, in _decorator
    result = func(self, *args, **kwargs)
  File "/Users/koen/workspace/PySyft/packages/syft/src/syft/service/request/request_service.py", line 217, in apply
    mark_as_read(context=context, uid=request_notification.id)
AttributeError: 'SyftError' object has no attribute 'id'

In [ ]:
request.code.output_history[0]

In [ ]:
# accept_res = request.accept_by_depositing_result(job_info.result.get())

# assert isinstance(accept_res, SyftSuccess)
# accept_res

In [ ]:
# Need to refresh Job because we overwrite result
job_high = client_high.jobs[0]

action_store_high = node_high.python_node.get_service("actionservice").store
blob_store_high = node_high.python_node.get_service(
    "blobstorageservice"
).stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_high.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_high.permissions[job_high.result.syft_blob_storage_entry_id]
)

## Sync back to low

In [ ]:
client_high.jobs[0]

In [ ]:
low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

high_state

In [ ]:
diff_state_2 = compare_states(low_state, high_state)

diff_state_2

In [ ]:
# syft absolute
from syft import ActionObject

In [ ]:
resolved_state_low, resolved_state_high = resolve(
    diff_state_2, decision="high", share_private_objects=True
)

In [ ]:
print(resolved_state_low)
print()
print(resolved_state_high)

In [ ]:
client_low.apply_state(resolved_state_low)

In [ ]:
client_high.apply_state(resolved_state_high)

In [ ]:
action_store_low = node_low.python_node.get_service("actionservice").store
blob_store_low = node_low.python_node.get_service("blobstorageservice").stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_low.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_low.permissions[job_high.result.syft_blob_storage_entry_id]
)

# Run code low

## Run

In [ ]:
client_low_ds._fetch_api(client_low.credentials)

In [ ]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low.get()

In [ ]:
res_low.get()

In [ ]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id == job_high.result.id.id == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [ ]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low.get()

In [ ]:
res_low.get()

In [ ]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id == job_high.result.id.id == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [ ]:
private_high.mean()

In [ ]:
job_low = client_low_ds.code.compute_mean(data=data_low, blocking=False)
job_low

In [ ]:
job_low.wait().get()

In [ ]:
job_low.logs()

In [ ]:
assert job_low.id == job_high.id
assert job_low.result.id == job_high.result.id
assert (
    job_low.result.syft_blob_storage_entry_id
    == job_high.result.syft_blob_storage_entry_id
)